In [1]:
import datetime, warnings, time
import numpy as np
import pandas as pd
import dask.array as da
import dask.dataframe as dd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
# from mpl_toolkits.basemap import Basemap
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

## Divinding the dataset into chunks

In [51]:
delays = {}
total_rows = 10915496
num_rows = 1000000
it = total_rows//num_rows
for i in range(it):
    s = time.time()
    delays["part_{}".format(i)] = pd.read_csv('../data/FlightDelays.csv', skiprows=i*num_rows, nrows=num_rows, low_memory=False)
    if i == 0:
        col_names = delays["part_{}".format(i)].columns
        delays["part_{}".format(i)] = delays["part_{}".format(i)].rename(columns={'Route': 'ROUTE'})
    else:
        delays["part_{}".format(i)].columns = col_names
        delays["part_{}".format(i)] = delays["part_{}".format(i)].rename(columns={'Route': 'ROUTE'})
    print("Rows {0} - {1} loaded in {2} s.".format(i*num_rows, (i+1)*num_rows, time.time()-s))
# Load last chunk
s = time.time()
delays["part_{}".format(it)] = pd.read_csv('../data/FlightDelays.csv', skiprows=it*num_rows, nrows=total_rows - it*num_rows, low_memory=False)
delays["part_{}".format(i)].columns = col_names
delays["part_{}".format(it)] = delays["part_{}".format(i)].rename(columns={'Route': 'ROUTE'})
print("Rows {0} - {1} loaded in {2} s.".format(it*num_rows, total_rows, time.time()-s))

Rows 0 - 1000000 loaded in 7.96874213218689 s.
Rows 1000000 - 2000000 loaded in 9.593935251235962 s.
Rows 2000000 - 3000000 loaded in 10.09010100364685 s.
Rows 3000000 - 4000000 loaded in 10.0872802734375 s.
Rows 4000000 - 5000000 loaded in 11.099240779876709 s.
Rows 5000000 - 6000000 loaded in 11.751289129257202 s.
Rows 6000000 - 7000000 loaded in 12.935537099838257 s.
Rows 7000000 - 8000000 loaded in 15.804089069366455 s.
Rows 8000000 - 9000000 loaded in 15.713647842407227 s.
Rows 9000000 - 10000000 loaded in 16.99952006340027 s.
Rows 10000000 - 10915496 loaded in 18.568474054336548 s.


## Pick a data chunk to design data cleaning operations

1. Change column types to categorical, numeric, and datetime
2. Perform imputation to fill missing values
    * Grouped by airport, airline, year, quarter, month, day of month, day of week
3. Change all the `float64` columns except `NET_INCOME` and `OP_REVENUES` to `integer`after imputation

In [130]:
test = delays['part_1'].copy()

In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 50 columns):
YEAR                   1000000 non-null int64
QUARTER                1000000 non-null int64
MONTH                  1000000 non-null int64
DAY_OF_MONTH           1000000 non-null int64
DAY_OF_WEEK            1000000 non-null int64
FL_DATE                1000000 non-null object
CARRIER                1000000 non-null object
FL_NUM                 1000000 non-null int64
ROUTE                  1000000 non-null int64
ORIGIN                 1000000 non-null object
DEST                   1000000 non-null object
DEST_CITY              1000000 non-null object
DEST_STATE             1000000 non-null object
CRS_DEP_TIME           1000000 non-null int64
DEP_TIME               978593 non-null float64
DEP_DELAY              977326 non-null float64
DEP_DELAY_NEW          977326 non-null float64
DEP_DEL15              977326 non-null float64
DEP_DELAY_GROUP        977326 non-null float64
DEP

In [132]:
test.head().T

,0,1,2,3,4
YEAR,2018,2018,2018,2018,2018
QUARTER,1,1,1,1,1
MONTH,2,2,2,2,2
DAY_OF_MONTH,19,19,19,19,20
DAY_OF_WEEK,1,1,1,1,2
FL_DATE,2018-02-19,2018-02-19,2018-02-19,2018-02-19,2018-02-20
CARRIER,WN,WN,WN,WN,WN
FL_NUM,1095,1760,1074,212,1095
ROUTE,3625,3625,3625,3625,3625
ORIGIN,MCI,MCI,MCI,MCI,MCI


In [133]:
# Read description excel file
desc = pd.read_excel('../data/FlightDataDescription.xlsx')

In [134]:
def get_column_types(df):
    cat_, num_, dt_ = [], [], []
    for _, row in df.iterrows():
        if row['TYPE'] == 'Nominal' or row['TYPE'] == 'Ordinal' or row['TYPE'] == 'Binary':
            cat_.append(row['ATTRIBUTE'])
        if row['TYPE'] == 'Interval':
            num_.append(row['ATTRIBUTE'])
        if row['TYPE'] == 'yyyymmdd':
            dt_.append(row['ATTRIBUTE'])
    return cat_, num_, dt_

In [135]:
cats, nums, dts = get_column_types(desc)

In [136]:
# Add other date stats
cats.extend(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK'])

## Convert the categorical data

In [137]:
def convert_nan(df, cols):
    start = time.time()
    for col in cols:
        df[col] = df[col].fillna(-9999)
    print("Converted all null values to -9999 in {} s.".format(time.time() - start))

In [138]:
def convert_to_categorical(df, cols):
    start = time.time()
    for col in cols:
        df[col] = df[col].astype('category')
    print("Converted to categorical columns in {} s.".format(time.time() - start))

In [139]:
def convert_to_integer(df, cols):
    start = time.time()
    for col in cols:
        df[col] = pd.to_numeric(df[col], downcast='integer', errors='ignore')
    print("Converted to numeric columns in {} s.".format(time.time() - start))

In [140]:
def convert_to_datetime(df, cols):
    start = time.time()
    for col in cols:
        df[col] = pd.to_datetime(df[col])
    print("Converted to datetime columns in {} s.".format(time.time() - start))

In [141]:
convert_nan(test, cats + nums)

Converted all null values to -9999 in 1.0817339420318604 s.


In [142]:
convert_to_categorical(test, cats)

Converted to categorical columns in 2.336487054824829 s.


In [143]:
convert_to_integer(test, nums)

Converted to numeric columns in 2.293158769607544 s.


In [144]:
convert_to_datetime(test, dts)

Converted to datetime columns in 0.2363567352294922 s.


In [145]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 50 columns):
YEAR                   1000000 non-null category
QUARTER                1000000 non-null category
MONTH                  1000000 non-null category
DAY_OF_MONTH           1000000 non-null category
DAY_OF_WEEK            1000000 non-null category
FL_DATE                1000000 non-null datetime64[ns]
CARRIER                1000000 non-null object
FL_NUM                 1000000 non-null category
ROUTE                  1000000 non-null category
ORIGIN                 1000000 non-null category
DEST                   1000000 non-null category
DEST_CITY              1000000 non-null category
DEST_STATE             1000000 non-null category
CRS_DEP_TIME           1000000 non-null int16
DEP_TIME               1000000 non-null int16
DEP_DELAY              1000000 non-null int16
DEP_DELAY_NEW          1000000 non-null int16
DEP_DEL15              1000000 non-null category
DEP_DELAY_GROU

40% reduction in memory! It will be further reduced after imputation.

## Imputation

In [20]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
CRS_DEP_TIME,1000000.0,1.326464e+03,4.857899e+02,1.00,915.0,1320.00,1730.0,2359.0
DEP_TIME,976576.0,1.332650e+03,4.963033e+02,1.00,921.0,1328.00,1738.0,2400.0
DEP_DELAY,975611.0,9.477413e+00,4.632530e+01,-51.00,-6.0,-2.00,6.0,2482.0
DEP_DELAY_NEW,975611.0,1.292271e+01,4.519102e+01,0.00,0.0,0.00,6.0,2482.0
TAXI_OUT,976110.0,1.801321e+01,1.054316e+01,1.00,12.0,15.00,21.0,178.0
WHEELS_OFF,976110.0,1.358663e+03,4.963074e+02,1.00,938.0,1342.00,1753.0,2400.0
WHEELS_ON,975289.0,1.477881e+03,5.189692e+02,1.00,1058.0,1513.00,1912.0,2400.0
TAXI_IN,975289.0,7.540325e+00,5.819347e+00,1.00,4.0,6.00,9.0,258.0
CRS_ARR_TIME,1000000.0,1.494522e+03,5.107359e+02,1.00,1110.0,1521.00,1919.0,2400.0
ARR_TIME,975289.0,1.483272e+03,5.228509e+02,1.00,1103.0,1517.00,1917.0,2400.0


In [ ]:
test = convert